In [3]:
import pandas as pd
import numpy as np
import re 
import csv

In [40]:
facilities_df = pd.read_csv('../raw_data/facilities.csv',
        usecols=[
             #   "facname", "address", 
                 "boro", "zipcode", "latitude", "longitude",
                 "facdomain", "facgroup", "facsubgrp", "factype",
                 "optype"],
        dtype= {"zipcode": np.str})

facilities_df.head()

boro zipcode   latitude  longitude  \
0  Manhattan   10009  40.728802 -73.979284   
1     Queens   11368  40.753795 -73.865001   
2      Bronx   10461  40.852561 -73.839535   
3     Queens   11432  40.711657 -73.786888   
4  Manhattan   10013  40.718027 -74.006059   

                             facdomain                    facgroup  \
0            Health and Human Services                 Health Care   
1  Education, Child Welfare, and Youth  Child Services and Welfare   
2            Health and Human Services                 Health Care   
3            Health and Human Services                 Health Care   
4      Libraries and Cultural Programs       Cultural Institutions   

                     facsubgrp                                       factype  \
0                Mental Health                      Outpatient Mental Health   
1              Child Nutrition                      Summer Only Feeding Site   
2        Hospitals and Clinics                     Hospital Extension Clinic   
3          Chemical Dependency  Opioid Treatment Program Chemical Dependency   
4  Other Cultural Institutions                Multi-Discipl, Perf & Non-Perf   

       optype  
0  Non-public  
1  Non-public  
2  Non-public  
3  Non-public  
4  Non-public

In [3]:
facilities_df.dtypes

boro          object
zipcode       object
latitude     float64
longitude    float64
facdomain     object
facgroup      object
facsubgrp     object
factype       object
optype        object
dtype: object

In [4]:
len(facilities_df)

36112

In [5]:
# NYC coords:  latitude = 40.730610, longitude = -73.935242
#
# using 1 mile as a walking distance radius 
#   - it is about a 10 min walk (soource: https://en.wikipedia.org/wiki/Walking_distance_measure)
#

from math import sin, cos, sqrt, atan2, radians

# Haversine formula for geo distance
def geo_distance(lat1, long1, lat2, long2):
    R = 6373.0 # approximate radius of earth in km

    lat1_r = radians(lat1)
    lon1_r = radians(long1)
    lat2_r = radians(lat2)
    lon2_r = radians(long2)

    dlon_r = lon2_r - lon1_r
    dlat_r = lat2_r - lat1_r

    a = sin(dlat_r / 2)**2 + cos(lat1_r) * cos(lat2_r) * sin(dlon_r / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance_km = R * c
    distance_mi = distance_km * 0.621371

    # print("Result in km:", distance_km)
    # print("Result in mi:", distance_mi)

    return distance_mi

print(geo_distance(40.0, -73.0, 40.0115, -73.0115))
print(geo_distance(40.0, -73.0, 40.0, -73.019))
print(geo_distance(40.0, -73.0, 40.0145, -73.0))
print(geo_distance(40.0, -73.0, 40.0145, -73.01))

1.0012003045861244
1.0059579639364535
1.002167394203805
1.1334014771913552


In [6]:
# Subway stations coordinates data: station names with latitude and longitude coordinates.
stations_geo_df = pd.read_csv('../raw_data/DOITT_SUBWAY_STATION_01_13SEPT2010.csv',
        usecols=["NAME", "the_geom"])
stations_geo_df.head()

NAME                                      the_geom
0          Astor Pl  POINT (-73.99106999861966 40.73005400028978)
1          Canal St  POINT (-74.00019299927328 40.71880300107709)
2           50th St  POINT (-73.98384899986625 40.76172799961419)
3         Bergen St  POINT (-73.97499915116808 40.68086213682956)
4  Pennsylvania Ave  POINT (-73.89488591154061 40.66471445143568)

In [7]:
len(stations_geo_df)

473

In [8]:
# Clean up the stations coordinates data
stations_geo_df.rename(columns = {'NAME':'STATION'}, inplace = True)
stations_geo_df['STATION'] = stations_geo_df['STATION'].str.upper()

stations_geo_df['latitude'] = pd.to_numeric(stations_geo_df['the_geom'].str.extract('(-?\d*\.\d*(?=\)))', expand=False))
stations_geo_df['longitude'] = pd.to_numeric(stations_geo_df['the_geom'].str.extract('(-?\d*\.\d*)', expand=False))
stations_geo_df.drop(['the_geom'], axis=1, inplace=True)

stations_geo_df.head()

STATION   latitude  longitude
0          ASTOR PL  40.730054 -73.991070
1          CANAL ST  40.718803 -74.000193
2           50TH ST  40.761728 -73.983849
3         BERGEN ST  40.680862 -73.974999
4  PENNSYLVANIA AVE  40.664714 -73.894886

In [9]:
stations_geo_df.dtypes

STATION       object
latitude     float64
longitude    float64
dtype: object

In [10]:
# merge the facilities data onto the geo data
# Reference used: 
# https://stackoverflow.com/questions/42877802/pandas-dataframe-join-items-in-range-based-on-their-geo-coordinates-longitude

# Cartesian product merge
stations_geo_df['mergevar'] = 1
facilities_df['mergevar'] = 1

stations_geo_facilities = pd.merge(stations_geo_df,facilities_df,on='mergevar')
stations_geo_facilities.head()

STATION  latitude_x  longitude_x  mergevar       boro zipcode  latitude_y  \
0  ASTOR PL   40.730054    -73.99107         1  Manhattan   10009   40.728802   
1  ASTOR PL   40.730054    -73.99107         1     Queens   11368   40.753795   
2  ASTOR PL   40.730054    -73.99107         1      Bronx   10461   40.852561   
3  ASTOR PL   40.730054    -73.99107         1     Queens   11432   40.711657   
4  ASTOR PL   40.730054    -73.99107         1  Manhattan   10013   40.718027   

   longitude_y                            facdomain  \
0   -73.979284            Health and Human Services   
1   -73.865001  Education, Child Welfare, and Youth   
2   -73.839535            Health and Human Services   
3   -73.786888            Health and Human Services   
4   -74.006059      Libraries and Cultural Programs   

                     facgroup                    facsubgrp  \
0                 Health Care                Mental Health   
1  Child Services and Welfare              Child Nutrition   
2                 Health Care        Hospitals and Clinics   
3                 Health Care          Chemical Dependency   
4       Cultural Institutions  Other Cultural Institutions   

                                        factype      optype  
0                      Outpatient Mental Health  Non-public  
1                      Summer Only Feeding Site  Non-public  
2                     Hospital Extension Clinic  Non-public  
3  Opioid Treatment Program Chemical Dependency  Non-public  
4                Multi-Discipl, Perf & Non-Perf  Non-public

In [14]:
stations_geo_facilities.iloc[0].latitude_x

# try the distance formula
dist0 = geo_distance(stations_geo_facilities.iloc[0].latitude_x, stations_geo_facilities.iloc[0].longitude_x, 
                     stations_geo_facilities.iloc[0].latitude_y, stations_geo_facilities.iloc[0].longitude_y)
dist0

dist1 = geo_distance(stations_geo_facilities.iloc[1].latitude_x, stations_geo_facilities.iloc[1].longitude_x, 
                     stations_geo_facilities.iloc[1].latitude_y, stations_geo_facilities.iloc[1].longitude_y)
dist1

6.802523981575836

In [15]:
len(stations_geo_facilities)

17080976

In [24]:
test_df = stations_geo_facilities.iloc[0:1000000].copy()
test_df['dist'] = test_df.apply(lambda row: geo_distance(row['latitude_x'], 
                                            row['longitude_x'], 
                                            row['latitude_y'], 
                                            row['longitude_y']), axis=1)
test_df.head()

STATION  latitude_x  longitude_x  mergevar       boro zipcode  latitude_y  \
0  ASTOR PL   40.730054    -73.99107         1  Manhattan   10009   40.728802   
1  ASTOR PL   40.730054    -73.99107         1     Queens   11368   40.753795   
2  ASTOR PL   40.730054    -73.99107         1      Bronx   10461   40.852561   
3  ASTOR PL   40.730054    -73.99107         1     Queens   11432   40.711657   
4  ASTOR PL   40.730054    -73.99107         1  Manhattan   10013   40.718027   

   longitude_y                            facdomain  \
0   -73.979284            Health and Human Services   
1   -73.865001  Education, Child Welfare, and Youth   
2   -73.839535            Health and Human Services   
3   -73.786888            Health and Human Services   
4   -74.006059      Libraries and Cultural Programs   

                     facgroup                    facsubgrp  \
0                 Health Care                Mental Health   
1  Child Services and Welfare              Child Nutrition   
2                 Health Care        Hospitals and Clinics   
3                 Health Care          Chemical Dependency   
4       Cultural Institutions  Other Cultural Institutions   

                                        factype      optype       dist  
0                      Outpatient Mental Health  Non-public   0.623331  
1                      Summer Only Feeding Site  Non-public   6.802524  
2                     Hospital Extension Clinic  Non-public  11.600218  
3  Opioid Treatment Program Chemical Dependency  Non-public  10.770782  
4                Multi-Discipl, Perf & Non-Perf  Non-public   1.143407

In [27]:
for i in range(0, 17):
    test_df = stations_geo_facilities.iloc[i*1000000:(i+1)*1000000].copy()
    test_df['dist'] = test_df.apply(lambda row: geo_distance(row['latitude_x'], 
                                                row['longitude_x'], 
                                                row['latitude_y'], 
                                                row['longitude_y']), axis=1)

    test_df.to_csv('stations_geo_facilities_'+ str(i*1000000)+'_'+str((i+1)*1000000)+'.csv')

In [51]:
test_df = stations_geo_facilities.iloc[17*1000000:].copy()
test_df['dist'] = test_df.apply(lambda row: geo_distance(row['latitude_x'], 
                                            row['longitude_x'], 
                                            row['latitude_y'], 
                                            row['longitude_y']), axis=1)

test_df.to_csv('stations_geo_facilities_'+ str(17*1000000)+'_max.csv')

In [29]:
test_df.head()

STATION  latitude_x  longitude_x  mergevar       boro zipcode  \
17000000  72ND ST   40.768803   -73.958362         1   Brooklyn   11219   
17000001  72ND ST   40.768803   -73.958362         1      Bronx   10457   
17000002  72ND ST   40.768803   -73.958362         1     Queens   11433   
17000003  72ND ST   40.768803   -73.958362         1   Brooklyn   11223   
17000004  72ND ST   40.768803   -73.958362         1  Manhattan   10009   

          latitude_y  longitude_y                             facdomain  \
17000000   40.640279   -73.994508  Parks, Gardens, and Historical Sites   
17000001   40.846494   -73.897603   Education, Child Welfare, and Youth   
17000002   40.697889   -73.786271  Parks, Gardens, and Historical Sites   
17000003   40.605689   -73.986230       Libraries and Cultural Programs   
17000004   40.723499   -73.976565   Education, Child Welfare, and Youth   

                                 facgroup  \
17000000                 Parks and Plazas   
17000001  Child Care and Pre-Kindergarten   
17000002                 Parks and Plazas   
17000003                        Libraries   
17000004                   Youth Services   

                                                  facsubgrp  \
17000000                    Streetscapes, Plazas, and Malls   
17000001                     DOE Universal Pre-Kindergarten   
17000002                    Streetscapes, Plazas, and Malls   
17000003                                   Public Libraries   
17000004  Youth Centers, Literacy Programs, Job Training...   

                          factype      optype       dist  
17000000           Triangle/Plaza      Public   9.082549  
17000001  Early Education Program  Non-public   6.239877  
17000002           Triangle/Plaza      Public  10.259224  
17000003         Public Libraries  Non-public  11.367803  
17000004            Youth Program  Non-public   3.273014

In [52]:
test_df_min_dist = test_df.groupby(by=['STATION', 'latitude_x', 'longitude_x'], as_index=False).min()
test_df_min_dist.drop(['boro', 'facdomain', 'facgroup', 'facsubgrp', 'factype', 'latitude_y', 'longitude_y', 'mergevar', 'optype'], axis=1, inplace=True)


In [53]:
print(len(test_df))
print(len(test_df_min_dist))

80976
3


In [55]:
test_df.head()

STATION  latitude_x  longitude_x  mergevar       boro zipcode  \
17000000  72ND ST   40.768803   -73.958362         1   Brooklyn   11219   
17000001  72ND ST   40.768803   -73.958362         1      Bronx   10457   
17000002  72ND ST   40.768803   -73.958362         1     Queens   11433   
17000003  72ND ST   40.768803   -73.958362         1   Brooklyn   11223   
17000004  72ND ST   40.768803   -73.958362         1  Manhattan   10009   

          latitude_y  longitude_y                             facdomain  \
17000000   40.640279   -73.994508  Parks, Gardens, and Historical Sites   
17000001   40.846494   -73.897603   Education, Child Welfare, and Youth   
17000002   40.697889   -73.786271  Parks, Gardens, and Historical Sites   
17000003   40.605689   -73.986230       Libraries and Cultural Programs   
17000004   40.723499   -73.976565   Education, Child Welfare, and Youth   

                                 facgroup  \
17000000                 Parks and Plazas   
17000001  Child Care and Pre-Kindergarten   
17000002                 Parks and Plazas   
17000003                        Libraries   
17000004                   Youth Services   

                                                  facsubgrp  \
17000000                    Streetscapes, Plazas, and Malls   
17000001                     DOE Universal Pre-Kindergarten   
17000002                    Streetscapes, Plazas, and Malls   
17000003                                   Public Libraries   
17000004  Youth Centers, Literacy Programs, Job Training...   

                          factype      optype       dist  
17000000           Triangle/Plaza      Public   9.082549  
17000001  Early Education Program  Non-public   6.239877  
17000002           Triangle/Plaza      Public  10.259224  
17000003         Public Libraries  Non-public  11.367803  
17000004            Youth Program  Non-public   3.273014

In [54]:
test_df_min_dist.head()

STATION  latitude_x  longitude_x      dist
0  72ND ST   40.768803   -73.958362  0.074635
1  86TH ST   40.777861   -73.951771  0.033577
2  96TH ST   40.784237   -73.947066  0.027367

In [57]:
test_df_min_dist_merged = pd.merge(test_df, test_df_min_dist, on=['STATION', 'latitude_x', 'longitude_x', 'dist'])
test_df_min_dist_merged.head()

STATION  latitude_x  longitude_x  mergevar       boro zipcode  latitude_y  \
0  72ND ST   40.768803   -73.958362         1  Manhattan   10021   40.768737   
1  86TH ST   40.777861   -73.951771         1  Manhattan   10028   40.778345   
2  86TH ST   40.777861   -73.951771         1  Manhattan   10028   40.778345   
3  96TH ST   40.784237   -73.947066         1  Manhattan   10128   40.783856   

   longitude_y                               facdomain          facgroup  \
0   -73.959785     Education, Child Welfare, and Youth  Higher Education   
1   -73.951827  Core Infrastructure and Transportation    Transportation   
2   -73.951827    Parks, Gardens, and Historical Sites  Parks and Plazas   
3   -73.946922    Parks, Gardens, and Historical Sites  Parks and Plazas   

                      facsubgrp                       factype      optype  \
0      Colleges or Universities            4-Year Independent  Non-public   
1      Parking Lots and Garages             Commercial Garage  Non-public   
2  Privately Owned Public Space  Privately Owned Public Space  Non-public   
3  Privately Owned Public Space  Privately Owned Public Space  Non-public   

       dist  
0  0.074635  
1  0.033577  
2  0.033577  
3  0.027367

In [69]:
#testing
i = 0
csv_name = 'stations_geo_facilities_' + str(i*1000000) + '_' + str((i+1)*1000000) + '.csv'
chunk_df = pd.read_csv(csv_name, encoding = "ISO-8859-1")
chunk_df.head()

Unnamed: 0   STATION  latitude_x  longitude_x  mergevar       boro  \
0           0  ASTOR PL   40.730054    -73.99107         1  Manhattan   
1           1  ASTOR PL   40.730054    -73.99107         1     Queens   
2           2  ASTOR PL   40.730054    -73.99107         1      Bronx   
3           3  ASTOR PL   40.730054    -73.99107         1     Queens   
4           4  ASTOR PL   40.730054    -73.99107         1  Manhattan   

   zipcode  latitude_y  longitude_y                            facdomain  \
0  10009.0   40.728802   -73.979284            Health and Human Services   
1  11368.0   40.753795   -73.865001  Education, Child Welfare, and Youth   
2  10461.0   40.852561   -73.839535            Health and Human Services   
3  11432.0   40.711657   -73.786888            Health and Human Services   
4  10013.0   40.718027   -74.006059      Libraries and Cultural Programs   

                     facgroup                    facsubgrp  \
0                 Health Care                Mental Health   
1  Child Services and Welfare              Child Nutrition   
2                 Health Care        Hospitals and Clinics   
3                 Health Care          Chemical Dependency   
4       Cultural Institutions  Other Cultural Institutions   

                                        factype      optype       dist  
0                      Outpatient Mental Health  Non-public   0.623331  
1                      Summer Only Feeding Site  Non-public   6.802524  
2                     Hospital Extension Clinic  Non-public  11.600218  
3  Opioid Treatment Program Chemical Dependency  Non-public  10.770782  
4                Multi-Discipl, Perf & Non-Perf  Non-public   1.143407

In [5]:
# set i=18
for i in range(0, 18):
    #print("i=" + str(i))
    # read in csv into a dataframe
    if i == 17:
        chunk_df = pd.read_csv('stations_geo_facilities_'+ str(i*1000000)+'_max.csv', encoding = "ISO-8859-1", dtype= {"zipcode": np.str})
    else:
        csv_name = 'stations_geo_facilities_' + str(i*1000000) + '_' + str((i+1)*1000000) + '.csv'
        chunk_df = pd.read_csv(csv_name, encoding = "ISO-8859-1", dtype= {"zipcode": np.str})
    
    chunk_df.drop(['Unnamed: 0'], axis=1, inplace=True)

 #   print(chunk_df.head())
    
    # find the row with the min(dist) per subway latitude and longitude, and keep only that row
    chunk_df_min_dist = chunk_df.groupby(by=['STATION', 'latitude_x', 'longitude_x'], as_index=False).min()
    chunk_df_min_dist.drop(['boro', 'facdomain', 'facgroup', 'facsubgrp', 'factype', 'latitude_y', 'longitude_y', 'mergevar', 'optype'], axis=1, inplace=True)
    chunk_df_min_dist_merged = pd.merge(chunk_df, chunk_df_min_dist, on=['STATION', 'latitude_x', 'longitude_x', 'dist'])
    chunk_df_min_dist_merged.drop(['dist', 'latitude_y', 'longitude_y', 'mergevar'], axis=1, inplace=True)
    
  #  print(chunk_df_min_dist.head())
    
    #attach the resulting chunk df to stations_geo_facilities_min_dist
    if i == 0:
        # stations_geo_facilities_zipcodes = chunk_df.copy()
        stations_geo_facilities_zipcodes = chunk_df_min_dist_merged.copy()
    else:
        # attach chunk_df to stations_geo_facilities_zipcodes
        stations_geo_facilities_zipcodes = pd.concat([stations_geo_facilities_zipcodes, chunk_df_min_dist_merged])
      

In [90]:
stations_geo_facilities_zipcodes.head()

STATION  latitude_x  longitude_x       boro zipcode  \
0   ASTOR PL   40.730054   -73.991070  Manhattan   10003   
1   CANAL ST   40.718803   -74.000193  Manhattan   10013   
2    50TH ST   40.761728   -73.983849  Manhattan   10019   
3    50TH ST   40.761728   -73.983849  Manhattan   10019   
4  BERGEN ST   40.680862   -73.974999   Brooklyn   11217   

                              facdomain               facgroup  \
0  Parks, Gardens, and Historical Sites       Historical Sites   
1       Libraries and Cultural Programs  Cultural Institutions   
2       Libraries and Cultural Programs  Cultural Institutions   
3       Libraries and Cultural Programs  Cultural Institutions   
4       Libraries and Cultural Programs  Cultural Institutions   

                     facsubgrp                         factype      optype  
0             Historical Sites            State Historic Place  Non-public  
1  Other Cultural Institutions  Multi-Discipl, Perf & Non-Perf  Non-public  
2  Other Cultural Institutions                         Theater  Non-public  
3  Other Cultural Institutions                           Music  Non-public  
4  Other Cultural Institutions                           Dance  Non-public

In [93]:
# remove duplicates
stations_geo_facilities_zipcodes.drop_duplicates(subset=['STATION', 'latitude_x', 'longitude_x'], keep='first', inplace=True)

In [111]:
# output to csv
stations_geo_facilities_zipcodes.to_csv('stations_geo_facilities_zipcodes.csv')

In [94]:
stations_geo_facilities_zipcodes.head()
    

STATION  latitude_x  longitude_x       boro zipcode  \
0          ASTOR PL   40.730054   -73.991070  Manhattan   10003   
1          CANAL ST   40.718803   -74.000193  Manhattan   10013   
2           50TH ST   40.761728   -73.983849  Manhattan   10019   
4         BERGEN ST   40.680862   -73.974999   Brooklyn   11217   
5  PENNSYLVANIA AVE   40.664714   -73.894886   Brooklyn   11207   

                                           facdomain               facgroup  \
0               Parks, Gardens, and Historical Sites       Historical Sites   
1                    Libraries and Cultural Programs  Cultural Institutions   
2                    Libraries and Cultural Programs  Cultural Institutions   
4                    Libraries and Cultural Programs  Cultural Institutions   
5  Public Safety, Emergency Services, and Adminis...     Emergency Services   

                     facsubgrp                         factype      optype  
0             Historical Sites            State Historic Place  Non-public  
1  Other Cultural Institutions  Multi-Discipl, Perf & Non-Perf  Non-public  
2  Other Cultural Institutions                         Theater  Non-public  
4  Other Cultural Institutions                           Dance  Non-public  
5                Fire Services                       Firehouse      Public

In [95]:
# apply distance formula -- DONT RUN, TAKES TOO LONG!
# stations_geo_facilities['dist'] = stations_geo_facilities.apply(lambda row: geo_distance(row['latitude_x'], 
 #                                           row['longitude_x'], 
 #                                           row['latitude_y'], 
 #                                           row['longitude_y']), axis=1)
# stations_geo_facilities.to_csv('stations_geo_facilities.csv')
# stations_geo_facilities.head()

In [96]:
mta_2017_df = pd.read_csv('../mta_data/year2017.csv')
mta_2017_df.head()

Unnamed: 0   C/A  UNIT       SCP STATION LINENAME DIVISION        DATE  \
0           1  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/25/2017   
1           2  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/25/2017   
2           3  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/25/2017   
3           4  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/25/2017   
4           5  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/25/2017   

       TIME     DESC  ENTRIES  EXITS             DATETIME  
0  07:00:00  REGULAR       18     13  2017-02-25 07:00:00  
1  11:00:00  REGULAR       67    103  2017-02-25 11:00:00  
2  15:00:00  REGULAR      191     70  2017-02-25 15:00:00  
3  19:00:00  REGULAR      364     73  2017-02-25 19:00:00  
4  23:00:00  REGULAR      141     34  2017-02-25 23:00:00

In [97]:
len(mta_2017_df)

2753449

In [98]:
# try merging MTA data with geo coords, using station name
mta_2017_df['STATION'] = mta_2017_df['STATION'].str.strip()
mta_2017_df.head()

Unnamed: 0   C/A  UNIT       SCP STATION LINENAME DIVISION        DATE  \
0           1  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/25/2017   
1           2  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/25/2017   
2           3  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/25/2017   
3           4  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/25/2017   
4           5  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/25/2017   

       TIME     DESC  ENTRIES  EXITS             DATETIME  
0  07:00:00  REGULAR       18     13  2017-02-25 07:00:00  
1  11:00:00  REGULAR       67    103  2017-02-25 11:00:00  
2  15:00:00  REGULAR      191     70  2017-02-25 15:00:00  
3  19:00:00  REGULAR      364     73  2017-02-25 19:00:00  
4  23:00:00  REGULAR      141     34  2017-02-25 23:00:00

In [107]:

mta_2017_geo_df = pd.merge(mta_2017_df, stations_geo_facilities_zipcodes, on='STATION', how='left', indicator=True)
mta_2017_geo_df

Unnamed: 0    C/A  UNIT       SCP        STATION LINENAME DIVISION  \
0                 1   A002  R051  02-00-00          59 ST  NQR456W      BMT   
1                 2   A002  R051  02-00-00          59 ST  NQR456W      BMT   
2                 3   A002  R051  02-00-00          59 ST  NQR456W      BMT   
3                 4   A002  R051  02-00-00          59 ST  NQR456W      BMT   
4                 5   A002  R051  02-00-00          59 ST  NQR456W      BMT   
5                 6   A002  R051  02-00-00          59 ST  NQR456W      BMT   
6                 7   A002  R051  02-00-00          59 ST  NQR456W      BMT   
7                 8   A002  R051  02-00-00          59 ST  NQR456W      BMT   
8                 9   A002  R051  02-00-00          59 ST  NQR456W      BMT   
9                10   A002  R051  02-00-00          59 ST  NQR456W      BMT   
10               11   A002  R051  02-00-00          59 ST  NQR456W      BMT   
11               12   A002  R051  02-00-00          59 ST  NQR456W      BMT   
12               13   A002  R051  02-00-00          59 ST  NQR456W      BMT   
13               14   A002  R051  02-00-00          59 ST  NQR456W      BMT   
14               15   A002  R051  02-00-00          59 ST  NQR456W      BMT   
15               16   A002  R051  02-00-00          59 ST  NQR456W      BMT   
16               17   A002  R051  02-00-00          59 ST  NQR456W      BMT   
17               18   A002  R051  02-00-00          59 ST  NQR456W      BMT   
18               19   A002  R051  02-00-00          59 ST  NQR456W      BMT   
19               20   A002  R051  02-00-00          59 ST  NQR456W      BMT   
20               21   A002  R051  02-00-00          59 ST  NQR456W      BMT   
21               22   A002  R051  02-00-00          59 ST  NQR456W      BMT   
22               23   A002  R051  02-00-00          59 ST  NQR456W      BMT   
23               24   A002  R051  02-00-00          59 ST  NQR456W      BMT   
24               25   A002  R051  02-00-00          59 ST  NQR456W      BMT   
25               26   A002  R051  02-00-00          59 ST  NQR456W      BMT   
26               27   A002  R051  02-00-00          59 ST  NQR456W      BMT   
27               28   A002  R051  02-00-00          59 ST  NQR456W      BMT   
28               29   A002  R051  02-00-00          59 ST  NQR456W      BMT   
29               30   A002  R051  02-00-00          59 ST  NQR456W      BMT   
...             ...    ...   ...       ...            ...      ...      ...   
3374217      195928  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374218      195929  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374219      195930  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374220      195931  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374221      195932  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374222      195933  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374223      195934  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374224      195935  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374225      195936  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374226      195937  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374227      195938  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374228      195939  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374229      195940  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374230      195941  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374231      195942  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374232      195943  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374233      195944  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374234      195945  TRAM2  R469  00-05-01  RIT-ROOSEVELT        R      RIT   
3374235      195946  TRAM2  R469  00-05-01  RIT-ROOSEVELT  

In [103]:
mta_2017_geo_df.dtypes

Unnamed: 0        int64
C/A              object
UNIT             object
SCP              object
STATION          object
LINENAME         object
DIVISION         object
DATE             object
TIME             object
DESC             object
ENTRIES           int64
EXITS             int64
DATETIME         object
latitude_x      float64
longitude_x     float64
boro             object
zipcode          object
facdomain        object
facgroup         object
facsubgrp        object
factype          object
optype           object
_merge         category
dtype: object

In [108]:
len(mta_2017_geo_df)

3374247

In [110]:
len(mta_2017_df)

2753449

In [106]:
mta_2017_geo_df['_merge'].unique()

[both]
Categories (1, object): [both]

In [4]:
# read in top 30 station name 
top_30_station_names_mapping_df = pd.read_csv('station_name_mapping_top_30.csv')
top_30_station_names_mapping_df.head()

geodata_station_name mta_data_station_name
0                  59TH ST                 59 ST
1   34TH ST - PENN STATION        34 ST-PENN STA
2  GRAND CENTRAL - 42ND ST       GRD CNTRL-42 ST
3      HERALD SQ - 34TH ST       34 ST-HERALD SQ
4       UNION SQ - 14TH ST        14 ST-UNION SQ

In [8]:

top_30_stations_zipcodes_df = pd.merge(top_30_station_names_mapping_df, stations_geo_facilities_zipcodes, \
                                         left_on='geodata_station_name', right_on='STATION', how='left', indicator=True)

In [16]:
top_30_stations_zipcodes_df

geodata_station_name mta_data_station_name  \
0                              59TH ST                 59 ST   
1               34TH ST - PENN STATION        34 ST-PENN STA   
2               34TH ST - PENN STATION        34 ST-PENN STA   
3               34TH ST - PENN STATION        34 ST-PENN STA   
4               34TH ST - PENN STATION        34 ST-PENN STA   
5              GRAND CENTRAL - 42ND ST       GRD CNTRL-42 ST   
6              GRAND CENTRAL - 42ND ST       GRD CNTRL-42 ST   
7              GRAND CENTRAL - 42ND ST       GRD CNTRL-42 ST   
8              GRAND CENTRAL - 42ND ST       GRD CNTRL-42 ST   
9              GRAND CENTRAL - 42ND ST       GRD CNTRL-42 ST   
10             GRAND CENTRAL - 42ND ST       GRD CNTRL-42 ST   
11                 HERALD SQ - 34TH ST       34 ST-HERALD SQ   
12                 HERALD SQ - 34TH ST       34 ST-HERALD SQ   
13                  UNION SQ - 14TH ST        14 ST-UNION SQ   
14                  UNION SQ - 14TH ST        14 ST-UNION SQ   
15                  UNION SQ - 14TH ST        14 ST-UNION SQ   
16                  UNION SQ - 14TH ST        14 ST-UNION SQ   
17                  UNION SQ - 14TH ST        14 ST-UNION SQ   
18                  UNION SQ - 14TH ST        14 ST-UNION SQ   
19                             23RD ST                 23 ST   
20                             23RD ST                 23 ST   
21                             23RD ST                 23 ST   
22                             23RD ST                 23 ST   
23                             23RD ST                 23 ST   
24                             23RD ST                 23 ST   
25                  TIMES SQ - 42ND ST        TIMES SQ-42 ST   
26                  TIMES SQ - 42ND ST        TIMES SQ-42 ST   
27                  TIMES SQ - 42ND ST        TIMES SQ-42 ST   
28                  TIMES SQ - 42ND ST        TIMES SQ-42 ST   
29   42ND ST - PORT AUTHORITY BUS TERM       42 ST-PORT AUTH   
..                                 ...                   ...   
80                             28TH ST                 28 ST   
81                             28TH ST                 28 ST   
82                             28TH ST                 28 ST   
83                             28TH ST                 28 ST   
84          JACKSON HTS - ROOSEVELT AV       JKSN HT-ROOSVLT   
85                             50TH ST                 50 ST   
86                             50TH ST                 50 ST   
87                             50TH ST                 50 ST   
88                             50TH ST                 50 ST   
89    W 4TH ST - WASHINGTON SQ (LOWER)        W 4 ST-WASH SQ   
90    W 4TH ST - WASHINGTON SQ (LOWER)        W 4 ST-WASH SQ   
91                             72ND ST                 72 ST   
92                             72ND ST                 72 ST   
93                             72ND ST                 72 ST   
94                             72ND ST                 72 ST   
95                             72ND ST                 72 ST   
96      ATLANTIC AV - BARCLAY'S CENTER        ATL AV-BARCLAY   
97      ATLANTIC AV - BARCLAY'S CENTER        ATL AV-BARCLAY   
98      ATLANTIC AV - BARCLAY'S CENTER        ATL AV-BARCLAY   
99                 42ND ST - BRYANT PK       42 ST-BRYANT PK   
100                            WALL ST               WALL ST   
101                            WALL ST               WALL ST   
102                            WALL ST               WALL ST   
103                            WALL ST               WALL ST   
104            LEXINGTON AVE - 53RD ST       LEXINGTON AV/53   
105                            7TH AVE                  7 AV   
106                            7TH AVE                  7 AV   
107                            7TH AVE                  7 AV   
108                            7TH AVE                  7 AV   
109                            7TH AVE                  7 AV   

                               STATION  latitude_

In [21]:
top_30_stations_zipcodes_df_lookup = top_30_stations_zipcodes_df.loc[:,['mta_data_station_name', 'boro', 'zipcode']]
top_30_stations_zipcodes_df_lookup.drop_duplicates(subset=['mta_data_station_name'], keep='first', inplace=True)
top_30_stations_zipcodes_df_lookup

mta_data_station_name       boro zipcode
0                   59 ST   Brooklyn   11220
1          34 ST-PENN STA  Manhattan   10001
5         GRD CNTRL-42 ST  Manhattan   10017
11        34 ST-HERALD SQ  Manhattan   10001
13         14 ST-UNION SQ  Manhattan   10003
19                  23 ST  Manhattan   10010
25         TIMES SQ-42 ST  Manhattan   10018
29        42 ST-PORT AUTH  Manhattan   10036
30              FULTON ST   Brooklyn   11217
37                  86 ST  Manhattan   10024
46         59 ST COLUMBUS  Manhattan   10023
48                 125 ST  Manhattan   10027
54               CANAL ST  Manhattan   10013
60                  96 ST  Manhattan   10128
64         47-50 STS ROCK  Manhattan   10020
65                  14 ST  Manhattan   10011
70          FLUSHING-MAIN     Queens   11354
71            CHAMBERS ST  Manhattan   10007
74           PATH NEW WTC  Manhattan   10007
75                  28 ST  Manhattan   10001
84        JKSN HT-ROOSVLT     Queens   11377
85                  50 ST  Manhattan   10019
89         W 4 ST-WASH SQ  Manhattan   10014
91                  72 ST  Manhattan   10023
96         ATL AV-BARCLAY   Brooklyn   11217
99        42 ST-BRYANT PK  Manhattan   10018
100               WALL ST  Manhattan   10005
104       LEXINGTON AV/53  Manhattan   10022
105                  7 AV   Brooklyn   11218

In [22]:
# output to csv
top_30_stations_zipcodes_df_lookup.to_csv('top_30_stations_zipcodes_df_lookup.csv')

In [33]:
# import income data
income_data_df = pd.read_excel('../income_data/income_data.xlsx',
  #      usecols=[
             #   "facname", "address", 
   #              "boro", "zipcode", "latitude", "longitude",
    #             "facdomain", "facgroup", "facsubgrp", "factype",
     #            "optype"],
        dtype= {"Zip Code": np.str})
                           
income_data_df.head()

# Zip Code               Location                City  Population  \
0  1    10007  40.713973, -74.008336  New York, New York        3522   
1  2    10280  40.708501, -74.016828  New York, New York        6614   
2  3    10162  40.769298, -73.949942  New York, New York        1726   
3  4    10004  40.691870, -74.014109  New York, New York        1225   
4  5    10069  40.775551, -73.989689  New York, New York        1403   

   Avg. Income/H/hold National Rank  
0              112947          #132  
1              108536          #157  
2              108416          #158  
3              101868          #238  
4               88091          #507

In [34]:
income_data_df.dtypes

#                      int64
Zip Code              object
Location              object
City                  object
Population             int64
Avg. Income/H/hold     int64
National Rank         object
dtype: object

In [36]:

top_30_stations_zipcodes_income_df = pd.merge(income_data_df, top_30_stations_zipcodes_df_lookup,\
                                              left_on = 'Zip Code', right_on = 'zipcode', how='inner', indicator=True)
top_30_stations_zipcodes_income_df

# Zip Code               Location                City  Population  \
0    1    10007  40.713973, -74.008336  New York, New York        3522   
1    1    10007  40.713973, -74.008336  New York, New York        3522   
2    8    10022  40.758615, -73.967752  New York, New York       30642   
3    9    10005  40.705390, -74.008988  New York, New York         884   
4   10    10024  40.786307, -73.970694  New York, New York       61414   
5   13    10023  40.775394, -73.982197  New York, New York       62206   
6   13    10023  40.775394, -73.982197  New York, New York       62206   
7   14    10128  40.781656, -73.950281  New York, New York       59856   
8   15    10017  40.752292, -73.972393  New York, New York       16201   
9   16    10014  40.733471, -74.007098  New York, New York       32667   
10  18    10010  40.739122, -73.982363  New York, New York       26408   
11  19    10011  40.741844, -74.000711  New York, New York       46669   
12  20    10003  40.731856, -73.989145  New York, New York       53673   
13  22    10019  40.766331, -73.985013  New York, New York       36012   
14  25    10018  40.754969, -73.992126  New York, New York        4255   
15  25    10018  40.754969, -73.992126  New York, New York        4255   
16  26    10036  40.759614, -73.991148  New York, New York       18751   
17  27    10001  40.750722, -73.997276  New York, New York       17310   
18  27    10001  40.750722, -73.997276  New York, New York       17310   
19  27    10001  40.750722, -73.997276  New York, New York       17310   
20  29    10013  40.720067, -74.004829  New York, New York       25042   
21  38    10027  40.811407, -73.953564  New York, New York       56168   
22  48    10020  40.758236, -73.978833  New York, New York           1   

    Avg. Income/H/hold National Rank mta_data_station_name       boro zipcode  \
0               112947          #132           CHAMBERS ST  Manhattan   10007   
1               112947          #132          PATH NEW WTC  Manhattan   10007   
2                80406          #820       LEXINGTON AV/53  Manhattan   10022   
3                79517          #860               WALL ST  Manhattan   10005   
4                78066          #935                 86 ST  Manhattan   10024   
5                72424        #1,297        59 ST COLUMBUS  Manhattan   10023   
6                72424        #1,297                 72 ST  Manhattan   10023   
7                70031        #1,547                 96 ST  Manhattan   10128   
8                69273        #1,611       GRD CNTRL-42 ST  Manhattan   10017   
9                66601        #1,931        W 4 ST-WASH SQ  Manhattan   10014   
10               62467        #2,471                 23 ST  Manhattan   10010   
11               61986        #2,569                 14 ST  Manhattan   10011   
12               60891        #2,796        14 ST-UNION SQ  Manhattan   10003   
13               55869        #3,901                 50 ST  Manhattan   10019   
14               48705        #6,447        TIMES SQ-42 ST  Manhattan   10018   
15               48705        #6,447       42 ST-BRYANT PK  Manhattan   10018   
16               41002       #11,152       42 ST-PORT AUTH  Manhattan   10036   
17               40932       #11,228        34 ST-PENN STA  Manhattan   10001   
18               40932       #11,228       34 ST-HERALD SQ  Manhattan   10001   
19               40932       #11,228                 28 ST  Manhattan   10001   
20               38304       #13,643              CANAL ST  Manhattan   10013   
21               23150       #29,273                125 ST  Manhattan   10027   
22                   0       #31,901        47-50 STS ROCK  Manhattan   10020   

   _merge  
0    both  
1    both  
2    both  
3    both  
4    both  
5    both  
6    both  
7    both  
8    both  
9    both  
10   both  
11   both  
12   both  
13   both  
14   both  
15   both  
16   both  
17   both  
18   both  
19   both  
20   both  
21   both  
22   bo

In [37]:
len(top_30_stations_zipcodes_income_df)

23

In [38]:
# output to csv
top_30_stations_zipcodes_income_df.to_csv('top_30_stations_zipcodes_income_df.csv')

In [52]:
# educational facilities count per zipcode
# by vars: boro, zipcode
school_k12_df = facilities_df[(facilities_df['facgroup']=='Schools (K-12)')].copy()

school_k12_df.drop(['latitude', 'longitude', 'facdomain', 'facsubgrp', 'factype','optype'], axis=1, inplace=True)
    


In [53]:
school_k12_df

boro zipcode        facgroup
20            Queens   11426  Schools (K-12)
27          Brooklyn   11209  Schools (K-12)
29         Manhattan   10040  Schools (K-12)
32          Brooklyn   11219  Schools (K-12)
40            Queens   11419  Schools (K-12)
68          Brooklyn   11220  Schools (K-12)
69          Brooklyn   11230  Schools (K-12)
82            Queens   11106  Schools (K-12)
113           Queens   11101  Schools (K-12)
133           Queens   11375  Schools (K-12)
137            Bronx   10467  Schools (K-12)
138         Brooklyn   11213  Schools (K-12)
146        Manhattan   10011  Schools (K-12)
150        Manhattan   10003  Schools (K-12)
152           Queens   11433  Schools (K-12)
154    Staten Island   10301  Schools (K-12)
160        Manhattan   10028  Schools (K-12)
164            Bronx   10459  Schools (K-12)
179         Brooklyn   11230  Schools (K-12)
207            Bronx   10467  Schools (K-12)
211         Brooklyn   11201  Schools (K-12)
213            Bronx   10456  Schools (K-12)
215           Queens   11368  Schools (K-12)
222         Brooklyn   11205  Schools (K-12)
242            Bronx   10453  Schools (K-12)
252           Queens   11354  Schools (K-12)
257         Brooklyn   11235  Schools (K-12)
259            Bronx   10461  Schools (K-12)
274           Queens   11426  Schools (K-12)
288           Queens   11426  Schools (K-12)
...              ...     ...             ...
35825      Manhattan   10011  Schools (K-12)
35827          Bronx   10456  Schools (K-12)
35836      Manhattan   10023  Schools (K-12)
35845         Queens   11370  Schools (K-12)
35852       Brooklyn   11225  Schools (K-12)
35856      Manhattan   10027  Schools (K-12)
35858      Manhattan   10010  Schools (K-12)
35866       Brooklyn   11237  Schools (K-12)
35875          Bronx   10466  Schools (K-12)
35880         Queens   11434  Schools (K-12)
35919      Manhattan   10128  Schools (K-12)
35936          Bronx   10467  Schools (K-12)
35946       Brooklyn   11236  Schools (K-12)
35952       Brooklyn   11212  Schools (K-12)
35985       Brooklyn   11207  Schools (K-12)
35987         Queens   11101  Schools (K-12)
35994      Manhattan   10019  Schools (K-12)
35995          Bronx   10461  Schools (K-12)
35999       Brooklyn   11219  Schools (K-12)
36007      Manhattan   10011  Schools (K-12)
36011      Manhattan   10040  Schools (K-12)
36029      Manhattan   10065  Schools (K-12)
36052      Manhattan   10033  Schools (K-12)
36072       Brooklyn   11220  Schools (K-12)
36077       Brooklyn   11206  Schools (K-12)
36080         Queens   11418  Schools (K-12)
36081       Brooklyn   11206  Schools (K-12)
36082         Queens   11413  Schools (K-12)
36093       Brooklyn   11249  Schools (K-12)
36100         Queens   11426  Schools (K-12)

[3385 rows x 3 columns]

In [54]:

school_k12_df =  school_k12_df.groupby(by=['boro', 'zipcode'], as_index=False).count()

In [55]:
len(school_k12_df)

182

In [62]:
school_k12_df = school_k12_df.rename(columns={'facgroup': 'schools_k12_count'})

In [63]:

top_30_stations_zipcodes_k12schools_df = pd.merge(school_k12_df, top_30_stations_zipcodes_df_lookup,\
                                              on = ['boro','zipcode'], how='inner', indicator=True)
top_30_stations_zipcodes_k12schools_df

boro zipcode  schools_k12_count mta_data_station_name _merge
0    Brooklyn   11217                 21             FULTON ST   both
1    Brooklyn   11217                 21        ATL AV-BARCLAY   both
2    Brooklyn   11218                 30                  7 AV   both
3    Brooklyn   11220                 16                 59 ST   both
4   Manhattan   10001                 13        34 ST-PENN STA   both
5   Manhattan   10001                 13       34 ST-HERALD SQ   both
6   Manhattan   10001                 13                 28 ST   both
7   Manhattan   10003                 26        14 ST-UNION SQ   both
8   Manhattan   10005                  4               WALL ST   both
9   Manhattan   10007                  5           CHAMBERS ST   both
10  Manhattan   10007                  5          PATH NEW WTC   both
11  Manhattan   10010                 17                 23 ST   both
12  Manhattan   10011                 26                 14 ST   both
13  Manhattan   10013                  8              CANAL ST   both
14  Manhattan   10014                  8        W 4 ST-WASH SQ   both
15  Manhattan   10017                  4       GRD CNTRL-42 ST   both
16  Manhattan   10018                  3        TIMES SQ-42 ST   both
17  Manhattan   10018                  3       42 ST-BRYANT PK   both
18  Manhattan   10019                 24                 50 ST   both
19  Manhattan   10022                  5       LEXINGTON AV/53   both
20  Manhattan   10023                 28        59 ST COLUMBUS   both
21  Manhattan   10023                 28                 72 ST   both
22  Manhattan   10024                 29                 86 ST   both
23  Manhattan   10027                 34                125 ST   both
24  Manhattan   10036                 10       42 ST-PORT AUTH   both
25  Manhattan   10128                 22                 96 ST   both
26     Queens   11354                 16         FLUSHING-MAIN   both
27     Queens   11377                 15       JKSN HT-ROOSVLT   both

In [64]:
len(top_30_stations_zipcodes_k12schools_df)

28

In [65]:
# output to csv
top_30_stations_zipcodes_k12schools_df.to_csv('top_30_stations_zipcodes_k12schools_df.csv')

In [66]:
# universities
# educational facilities count per zipcode
# by vars: boro, zipcode
school_higherEd_df = facilities_df[(facilities_df['facgroup']=='Higher Education')].copy()

school_higherEd_df.drop(['latitude', 'longitude', 'facdomain', 'facsubgrp', 'factype','optype'], axis=1, inplace=True)
    


In [68]:
len(school_higherEd_df)

164

In [69]:
school_higherEd_df = school_higherEd_df.rename(columns={'facgroup': 'higher_ed_facilities_count'})

In [70]:
school_higherEd_df.head()

boro zipcode higher_ed_facilities_count
640       Bronx   10468           Higher Education
737      Queens   11367           Higher Education
1054  Manhattan   10019           Higher Education
1134   Brooklyn   11205           Higher Education
1197  Manhattan   10023           Higher Education

In [72]:

school_higherEd_df =  school_higherEd_df.groupby(by=['boro', 'zipcode'], as_index=False).count()

In [80]:
school_higherEd_top10 = school_higherEd_df.sort_values(['higher_ed_facilities_count'],ascending=[0]).head(10).copy()
school_higherEd_top10

boro zipcode  higher_ed_facilities_count
9    Brooklyn   11201                          11
27  Manhattan   10011                          10
22  Manhattan   10001                           8
40  Manhattan   10027                           7
25  Manhattan   10007                           7
23  Manhattan   10003                           6
50  Manhattan   10065                           5
0       Bronx   10451                           5
32  Manhattan   10018                           4
30  Manhattan   10016                           4

In [81]:
# output to csv
school_higherEd_top10.to_csv('school_higherEd_top10.csv')

In [82]:
# higher ed counts for each of the top 30 stations

top_30_stations_zipcodes_higherEd_df = pd.merge(school_higherEd_df, top_30_stations_zipcodes_df_lookup,\
                                              on = ['boro','zipcode'], how='inner', indicator=True)
top_30_stations_zipcodes_higherEd_df

boro zipcode  higher_ed_facilities_count mta_data_station_name _merge
0   Manhattan   10001                           8        34 ST-PENN STA   both
1   Manhattan   10001                           8       34 ST-HERALD SQ   both
2   Manhattan   10001                           8                 28 ST   both
3   Manhattan   10003                           6        14 ST-UNION SQ   both
4   Manhattan   10007                           7           CHAMBERS ST   both
5   Manhattan   10007                           7          PATH NEW WTC   both
6   Manhattan   10010                           3                 23 ST   both
7   Manhattan   10011                          10                 14 ST   both
8   Manhattan   10013                           3              CANAL ST   both
9   Manhattan   10017                           1       GRD CNTRL-42 ST   both
10  Manhattan   10018                           4        TIMES SQ-42 ST   both
11  Manhattan   10018                           4       42 ST-BRYANT PK   both
12  Manhattan   10019                           3                 50 ST   both
13  Manhattan   10020                           1        47-50 STS ROCK   both
14  Manhattan   10022                           3       LEXINGTON AV/53   both
15  Manhattan   10023                           2        59 ST COLUMBUS   both
16  Manhattan   10023                           2                 72 ST   both
17  Manhattan   10024                           1                 86 ST   both
18  Manhattan   10027                           7                125 ST   both
19  Manhattan   10036                           1       42 ST-PORT AUTH   both
20     Queens   11354                           1         FLUSHING-MAIN   both

In [83]:
# output to csv
top_30_stations_zipcodes_higherEd_df.to_csv('top_30_stations_zipcodes_higherEd_df.csv')